#### Title:          02_create_Vicon_benchmark_angles_for_cutting.ipynb
#### Description:    Create timeseries with Vicon benchmark angles
#### Authors:        Fabian Kahl and Maximilian Kapsecker

In [1]:
import numpy as np
import pandas as pd
import glob
import datetime
import os
from utils import vectors, calculate_angles, compute_vicon_angles, start_to_end, drop_unnecessary_elements, generate_vicon_column_set

path_to_csvs = '../data/non_area_fileformat/frameworks/VICON/'

# Column name mapping
mapping = {
    'leftShoulder': 'LSJC',
    'leftElbow': 'LEJC',
    'leftHand': 'LWJC',
    'leftHip': 'LHJC',
    'leftKnee': 'LKJC',
    'leftFoot': 'LAJC',
    'rightShoulder': 'RSJC',
    'rightElbow': 'REJC',
    'rightHand': 'RWJC',
    'rightHip': 'RHJC',
    'rightKnee': 'RKJC',
    'rightFoot': 'RAJC'
}

# Parameters
exercises = ['Front lunge', 'Jumping jacks', 'Lateral arm raise',
             'Leg extension crunch', 'Reverse fly', 'Side squat',
             'Single leg deadlift', 'Squat']
views = ['Frontal', 'Side']
angles_benchmark = pd.DataFrame(index=exercises, columns=views)
angles_benchmark.loc['Front lunge'] = [['rightHip', 'rightKnee', 'rightFoot'],
                                       ['rightHip', 'rightKnee', 'rightFoot']]
angles_benchmark.loc['Jumping jacks'] = [['leftHip', 'leftShoulder', 'leftElbow'],
                                         ['leftHip', 'leftShoulder', 'leftElbow']]
angles_benchmark.loc['Lateral arm raise'] = [['leftHip', 'leftShoulder', 'leftElbow'],
                                            ['leftHip', 'leftShoulder', 'leftElbow']]
angles_benchmark.loc['Leg extension crunch'] = [['rightShoulder', 'rightHip', 'rightKnee'],
                                                ['rightHip', 'rightKnee', 'rightFoot']]
angles_benchmark.loc['Reverse fly'] = [['rightShoulder', 'leftShoulder', 'leftElbow'],
                                      ['rightShoulder', 'leftShoulder', 'leftElbow']]
angles_benchmark.loc['Side squat'] = [['rightHip', 'rightKnee', 'rightFoot'],
                                     ['rightHip', 'rightKnee', 'rightFoot']]
angles_benchmark.loc['Single leg deadlift'] = [['leftShoulder', 'leftHip', 'leftKnee'],
                                              ['leftShoulder', 'leftHip', 'leftKnee']]
angles_benchmark.loc['Squat'] = [['rightHip', 'rightKnee', 'rightFoot'],
                                ['rightHip', 'rightKnee', 'rightFoot']]

In [2]:
cols_vicon = generate_vicon_column_set(mapping)

In [3]:
subjects = glob.glob(path_to_csvs+"*")
subjects = [item for item in subjects if os.path.isdir(item)]

for k in subjects:
    subject = k.split('/')[-1]

    files_subject_vicon = glob.glob(path_to_csvs+subject+'/*.csv')
    # Drop unwanted csv files
    files_subject_vicon = drop_unnecessary_elements(files_subject_vicon, exercises)
    print(k.split('/')[-1], 'has', len(files_subject_vicon), 'Vicon files.')

    # Iterate over all vicon files for one individual
    for j in files_subject_vicon:
        df = pd.DataFrame()
        temp_vicon = pd.read_csv(j, skiprows=2, low_memory=False)

        # Vicon csv files do not follow a strict table based format according the following editing becomes necessary
        remember = 'default'
        cols = []
        for i, m in enumerate(np.array(temp_vicon.columns)):
            temp = ''
            if 'Unnamed' in m:
                aa = remember + '_' + str(temp_vicon[m][0]) + '_' + str(temp_vicon[m][1])
            else:
                base = m.replace('Takashi:' , '')
                base = base.replace(':' , '')
                base = base.replace('Sub10' , '')
                base = base.replace('Sub1' , '')
                base = base.replace('Sub2' , '')
                base = base.replace('Sub3' , '')
                base = base.replace('Sub4' , '')
                base = base.replace('Sub5' , '')
                base = base.replace('Sub6' , '')
                base = base.replace('Sub7' , '')
                base = base.replace('Sub8' , '')
                base = base.replace('Sub9' , '')
                aa = base + '_' + str(temp_vicon[m][0]) + '_' + str(temp_vicon[m][1])
                remember = base
            cols.append(aa)
        temp_vicon.columns = cols
        temp_vicon = temp_vicon[cols_vicon]
        # new
        temp_vicon = temp_vicon.rename(columns={'default_Frame_nan': 'Frame'})
        temp_vicon = temp_vicon.drop([0, 1])
        starttime, endtime = start_to_end(j[:-3] + 'xcp')
        temp_vicon.Frame = (temp_vicon.Frame.astype(float) / 250.0) * 1000.0
        temp_vicon.Frame = [starttime + datetime.timedelta(milliseconds=fra) for fra in temp_vicon.Frame]
        # Videos and Vicon have a timely delay of 2 hours (= 7200 s)
        temp_vicon['timestamp_vicon'] = ((temp_vicon.Frame - pd.Timestamp("1970-01-01")) / pd.Timedelta('1s')) - 7200.0
        temp_vicon['timestamp_start_vicon'] = temp_vicon.iloc[0].timestamp_vicon
        temp_vicon['timestamp_stop_vicon'] = temp_vicon.iloc[-1].timestamp_vicon
        temp_vicon['timestamp'] = temp_vicon['timestamp_vicon'] - temp_vicon['timestamp_start_vicon']

        df = pd.concat([df, temp_vicon], ignore_index=True)

        df.sort_values(['timestamp'], inplace=True)
    
        exercise = j.rsplit('/', 1)[-1].rsplit('.', 1)[0].split(' ', 1)[1]
        for view in views:
            angles_vi, config = compute_vicon_angles(df, angles_benchmark[view][exercise], mapping)
            df_angle = df[['timestamp']].copy()
            df_angle['angle'] = angles_vi
            df_angle.to_csv('../data/Vicon_benchmark_angles/'+subject+'_'+exercise+'_'+view+'.csv', index=False)

03 has 8 Vicon files.
04 has 8 Vicon files.
05 has 8 Vicon files.
02 has 8 Vicon files.
10 has 8 Vicon files.
07 has 8 Vicon files.
09 has 8 Vicon files.
08 has 8 Vicon files.
01 has 8 Vicon files.
06 has 8 Vicon files.
